In [6]:
##pip install scikit-suprise

import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
home.head()

,A15I9QV4OG22A6,001212835X,5.0,1389657600
0,A1BNZ90HN39TYR,0205062040,5.0,1373846400
1,A9DUCBTL8K3E4,0205062040,5.0,1382659200
2,AA8O583KRYINX,0205062040,5.0,1372723200
3,A16CR5RCPXFMMA,0205062040,4.0,1364256000
4,A398X3UMHLCM93,0205062040,5.0,1368403200


In [7]:
home= pd.read_csv('ratings_Tools_and_Home_improvement.csv',names=['user','item','rating','timestamp'])

In [8]:
home.head()

,user,item,rating,timestamp
0,A15I9QV4OG22A6,001212835X,5.0,1389657600
1,A1BNZ90HN39TYR,0205062040,5.0,1373846400
2,A9DUCBTL8K3E4,0205062040,5.0,1382659200
3,AA8O583KRYINX,0205062040,5.0,1372723200
4,A16CR5RCPXFMMA,0205062040,4.0,1364256000


In [10]:
from surprise import Dataset,Reader, SVD
from surprise.model_selection.validation import cross_validate

In [11]:
reader= Reader()

In [12]:
data= Dataset.load_from_df(home[['user','item','rating']],reader)

In [13]:
data

In [14]:
svd=SVD()

In [15]:
cv=cross_validate(svd,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2267  1.2269  1.2248  1.2274  1.2275  1.2266  0.0010  
MAE (testset)     0.9470  0.9472  0.9453  0.9474  0.9467  0.9467  0.0007  
Fit time          85.22   96.07   130.40  125.55  168.12  121.07  29.09   
Test time         3.39    3.65    6.72    4.76    2.54    4.21    1.44    


In [16]:
cv

{'test_rmse': array([1.22669341, 1.22689147, 1.22475033, 1.22737765, 1.22750023]),
 'test_mae': array([0.94696746, 0.94716631, 0.94533655, 0.94740354, 0.94672917]),
 'fit_time': (85.21889114379883,
  96.07308983802795,
  130.39517211914062,
  125.55419707298279,
  168.12036609649658),
 'test_time': (3.3930270671844482,
  3.6460490226745605,
  6.722504138946533,
  4.7561421394348145,
  2.5428338050842285)}

In [17]:
training= data.build_full_trainset()

In [19]:
svd.fit(training)

In [20]:
products= pd.DataFrame({'item': np.unique(home['item'])}).reset_index().drop('index',axis=1)

In [21]:
len(products)

260659

In [22]:
product_sample= products.sample(frac=0.01)

In [23]:
product_sample

,item
252426,B00G5R4E3Q
16552,B0002YPZLI
129070,B003E36UJU
188150,B006SEMQEK
235215,B00C8O2BM6
...,...
232724,B00BRYE6PI
78731,B0018OFVEM
47272,B000KSW1SG
6727,B00004Z2WN


In [24]:
customer= home.user[0]

In [25]:
customer

'A15I9QV4OG22A6'

In [26]:
product_sample['prediction']= product_sample['item'].apply(lambda x: svd.predict(customer,x).est)

In [28]:
product_sample.sort_values(by='prediction',ascending=False)

,item,prediction
137500,B003V0IMZQ,5.000000
142510,B0041SMX4S,4.941520
177519,B005OLVDJC,4.924237
10052,B0000CEZ8X,4.874159
44000,B000JFJLWE,4.869977
...,...,...
84833,B001DD9ZFY,2.927977
215130,B009AO8MO8,2.894556
205233,B008BXD7QC,2.858577
243881,B00DV1LF9A,2.832995
